In [ ]:
%pip install chess

In [ ]:
import chess
import chess.svg
import random
from IPython.display import display, SVG
import sys


def escolher_cor():
    # Pergunta ao jogador se deseja ser as peças brancas ou pretas
    while True:
        cor_escolhida = input(
            "Escolha a cor (B ou P): ").lower()
        if cor_escolhida == 'b':
            return chess.WHITE
        elif cor_escolhida == 'p':
            return chess.BLACK
        else:
            print("Opção inválida. Tente novamente.")


def obter_melhor_movimento(tabuleiro, cor, profundidade=3, num_movimentos=10):
    movimentos_legais = list(tabuleiro.legal_moves)
    movimentos_selecionados = random.sample(
        movimentos_legais, min(num_movimentos, len(movimentos_legais)))

    melhor_movimento = None
    melhor_pontuacao = float('-inf')

    for movimento in movimentos_selecionados:
        tabuleiro.push(movimento)
        pontuacao = -negamax(tabuleiro, profundidade - 1, cor)
        tabuleiro.pop()

        # Atualiza o melhor movimento com base na pontuação
        if melhor_movimento is None or pontuacao > melhor_pontuacao:
            melhor_pontuacao = pontuacao
            melhor_movimento = movimento

    return melhor_movimento


def negamax(tabuleiro, profundidade, cor):
    if profundidade == 0 or tabuleiro.is_game_over():
        return avaliar_tabuleiro(tabuleiro, cor)

    movimentos_legais = list(tabuleiro.legal_moves)
    melhor_pontuacao = float('-inf')

    for movimento in movimentos_legais:
        tabuleiro.push(movimento)
        pontuacao = -negamax(tabuleiro, profundidade - 1, cor)
        tabuleiro.pop()

        # Atualiza a melhor pontuação
        if pontuacao > melhor_pontuacao:
            melhor_pontuacao = pontuacao

    return melhor_pontuacao


def avaliar_tabuleiro(tabuleiro, cor):
    contagem = sum(1 for _ in tabuleiro.pieces(chess.PAWN, cor))
    contagem += 3 * sum(1 for _ in tabuleiro.pieces(chess.KNIGHT, cor))
    contagem += 3 * sum(1 for _ in tabuleiro.pieces(chess.BISHOP, cor))
    contagem += 5 * sum(1 for _ in tabuleiro.pieces(chess.ROOK, cor))
    contagem += 9 * sum(1 for _ in tabuleiro.pieces(chess.QUEEN, cor))
    return contagem


def jogada_jogador(tabuleiro):
    while True:
        display(SVG(chess.svg.board(board=tabuleiro, size=400)))
        movimento_jogador = input(
            "Seu movimento (Ex:a2a3, ou 'ç' para sair): ")

        if movimento_jogador.lower() == 'ç':
            print("Jogo interrompido.")
            sys.exit()

        try:
            movimento = chess.Move.from_uci(movimento_jogador)
            if movimento in tabuleiro.legal_moves:
                return movimento
            else:
                print("Movimento inválido. Tente novamente.")
        except ValueError:
            print("Entrada inválida. Tente novamente.")


def jogada_ia(tabuleiro, profundidade, cor):
    # Obtém o melhor movimento da IA
    display(SVG(chess.svg.board(board=tabuleiro, size=400)))
    print("Movimento da IA:")
    return obter_melhor_movimento(tabuleiro, cor, profundidade)


def iniciar_jogo():
    resposta = input("Você quer iniciar o jogo? (s/n): ").lower()
    return resposta == 's'


def jogar_jogo():
    tabuleiro = chess.Board()
    profundidade = 3

    cor_jogador = escolher_cor()
    jogador_inicia = cor_jogador == chess.WHITE

    display(SVG(chess.svg.board(board=tabuleiro, size=400)))

    while not tabuleiro.is_game_over():
        # Vez do jogador
        if jogador_inicia:
            movimento_jogador = jogada_jogador(tabuleiro)
            tabuleiro.push(movimento_jogador)
        else:
            # Vez da IA
            movimento_ia = jogada_ia(tabuleiro, profundidade, cor_jogador)
            tabuleiro.push(movimento_ia)

        if tabuleiro.is_game_over():
            break

        # Alternar vez do jogador
        jogador_inicia = not jogador_inicia

    print("Fim do jogo")
    print("Resultado: " + tabuleiro.result())


# Iniciar o jogo
jogar_jogo()